In [2]:
from time import sleep
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import csv
import os

In [3]:
url="https://www.jumia.ma/catalog/?q=samsung+telephone"

In [4]:
import requests
def get_img(url_img,id):
    r = requests.get(url_img)
    if r.status_code == 200:
        img_container="image"+str(id)+".png"
        with open(img_container, 'wb') as f:
            f.write(r.content)
            os.chdir("c:\\Users\\papew\\OneDrive\\Bureau\\pyth")

In [5]:
def save(elements,page=1):
   os.mkdir(os.path.join(os.getcwd(),"page"+str(page)))
   id=1
   with open("samsungdata.csv","a",newline='') as data:
    writer=csv.writer(data)
    for element in elements:
        livraison="payante"
        line=[]
        imgs=element.find_elements(By.XPATH,'.//div[@class="img-c"]/img')
        if (len(imgs)==2):
            livraison="gratuite"
        imgurl=imgs[0].get_attribute('data-src')
        info=element.find_element(By.XPATH,'.//div[@class="info"]')
        nom=info.find_element(By.CLASS_NAME,"name").text.replace('"',"")
        prix=info.find_element(By.CLASS_NAME,"prc").text
        line.append(nom)
        line.append(prix)
        os.chdir(os.path.join(os.getcwd(),"page"+str(page)))
        get_img(imgurl,id)
        id=id+1
        try:
          stars=info.find_element(By.XPATH,'.//div[@class="rev"]/div[@class="stars _s"]').text.replace("out of 5","")
          line.append(stars)
        except Exception as e:
          line.append("Nan")
        finally:
             line.append(livraison)
             writer.writerow(line)
      


In [6]:
options = uc.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = uc.Chrome(options=options)
driver.get(url)

sleep(10)

card=driver.find_element(By.CSS_SELECTOR,"section.card.-fh")
elements=card.find_elements(By.TAG_NAME,"article")
save(elements)

In [7]:
def find_next():
        pages=driver.find_element(By.CSS_SELECTOR,"div.pg-w.-ptm.-pbxl")
        pages=pages.find_elements(By.TAG_NAME,"a")
        suivant=None
        for page in pages:
                try:
                    if page.get_attribute("aria-label")=="Page suivante":
                          suivant=page.get_attribute("href")
                except Exception as StaleElementReferenceException:
                     pass
        return suivant

In [8]:
def scrap(url):
    driver.get(url)
    sleep(10)
    card=driver.find_element(By.CSS_SELECTOR,"section.card.-fh")
    elements=card.find_elements(By.TAG_NAME,"article")
    pages=driver.find_element(By.CSS_SELECTOR,"div.pg-w.-ptm.-pbxl")
    page=pages.find_element(By.CSS_SELECTOR,"a.pg._act").text
    save(elements,page)
    

In [9]:
suivant=find_next()
while suivant is not None:
    scrap(suivant)
    suivant=find_next()

In [10]:
driver.quit()